📂 결과 파일 (자동 생성됨)  

	01. data/secom_master_eda.csv : sensor_001 ~ sensor_590, timestamp, label  

	02. data/secom_model_train.csv : sensor_001 ~ sensor_590, label (timestamp 제거)  
	

# SECOM 데이터셋 합치기 (EDA용 & 모델링용)

이 노트북은 **SECOM 원본 데이터**(`secom.data`, `secom_labels.data`)를 불러와  
EDA 분석용과 모델 학습용 두 가지 CSV로 저장하는 역할을 한다.

- **EDA용 (secom_master_eda.csv)**  
  - 센서 피처(`sensor_001` ~ `sensor_590`) + `timestamp` + `label`  
  - 시간 패턴 / 결측 패턴 분석에 활용  

- **모델링용 (secom_model_train.csv)**  
  - 센서 피처(`sensor_001` ~ `sensor_590`) + `label`  
  - `timestamp` 제거 (데이터 누수 방지)  
  - 전처리 단계에서 라벨을 0/1로 변환하여 모델 학습에 활용  

### 경로 설정

In [1]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if (Path.cwd().name == "notebooks") else Path.cwd()
RAW_DIR = PROJECT_ROOT / "data" / "raw" / "secom"
OUT_DIR = PROJECT_ROOT / "data"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("RAW_DIR :", RAW_DIR)
print("OUT_DIR:", OUT_DIR)

RAW_DIR : /Users/mealkuo/Desktop/capstone02_project/data/raw/secom
OUT_DIR: /Users/mealkuo/Desktop/capstone02_project/data


### 원본 데이터 미리 보기

In [2]:
import pandas as pd

# 센서 피처 (X)
X_raw = pd.read_csv(RAW_DIR / "secom.data", sep=r"\s+", header=None, engine="python")
print("X_raw shape:", X_raw.shape)
display(X_raw.head(3))

# 라벨 + 타임스탬프 (y)
y_raw = pd.read_csv(RAW_DIR / "secom_labels.data", sep=r"\s+", header=None, engine="python")
print("y_raw shape:", y_raw.shape)
display(y_raw.head(3))

X_raw shape: (1567, 590)


,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602


y_raw shape: (1567, 3)


,0,1,2
0,-1,"""19/07/2008","11:55:00"""
1,-1,"""19/07/2008","12:32:00"""
2,1,"""19/07/2008","13:17:00"""


### 데이터셋 합치기

In [3]:
from datetime import datetime

def _parse_timestamp(ts: str):
    if pd.isna(ts): return pd.NaT
    ts = str(ts).replace('"','').strip()
    for fmt in ("%d/%m/%Y %H:%M:%S", "%Y-%m-%d %H:%M:%S", "%Y/%m/%d %H:%M:%S"):
        try: return datetime.strptime(ts, fmt)
        except Exception: continue
    return pd.NaT

def build_secom_master(X_raw, y_raw, out_dir: Path):
    # 1) 센서 피처 이름 붙이기
    X = X_raw.copy()
    X.columns = [f"sensor_{i:03d}" for i in range(1, X.shape[1]+1)]

    # 2) 라벨/타임스탬프 정리
    if y_raw.shape[1] == 3:
        y = pd.DataFrame({
            "label": y_raw.iloc[:,0],
            "timestamp": (y_raw.iloc[:,1].astype(str)+" "+y_raw.iloc[:,2].astype(str))
        })
    elif y_raw.shape[1] == 2:
        y = y_raw.copy()
        y.columns = ["label","timestamp"]
    else:
        raise ValueError(f"Unexpected labels shape: {y_raw.shape}")

    y["timestamp"] = y["timestamp"].map(_parse_timestamp)

    # 3) 합치기
    eda_df = pd.concat([X, y[["timestamp","label"]]], axis=1)

    # 4) 저장
    eda_path   = out_dir / "secom_master_eda.csv"   # EDA용 dataset
    model_path = out_dir / "secom_model_train.csv"  # Train용 dataset; model train 들어가기 전의 원본 클린 버전
    eda_df.to_csv(eda_path, index=False)
    eda_df.drop(columns=["timestamp"]).to_csv(model_path, index=False)

    return eda_df, eda_path, model_path

### 실행 및 결과 저장

In [4]:
eda_df, eda_path, model_path = build_secom_master(X_raw, y_raw, OUT_DIR)

print("[OK] Wrote:", eda_path)
print("[OK] Wrote:", model_path)
print("EDA shape :", eda_df.shape, "| MODEL shape :", (eda_df.shape[0], eda_df.shape[1]-1))
print("Label counts:", eda_df['label'].value_counts().to_dict())

[OK] Wrote: /Users/mealkuo/Desktop/capstone02_project/data/secom_master_eda.csv
[OK] Wrote: /Users/mealkuo/Desktop/capstone02_project/data/secom_model_train.csv
EDA shape : (1567, 592) | MODEL shape : (1567, 591)
Label counts: {-1: 1463, 1: 104}


### 최종 데이터 확인

In [5]:
eda_df.head(3)



,sensor_001,sensor_002,sensor_003,sensor_004,sensor_005,sensor_006,sensor_007,sensor_008,sensor_009,sensor_010,...,sensor_583,sensor_584,sensor_585,sensor_586,sensor_587,sensor_588,sensor_589,sensor_590,timestamp,label
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,2008-07-19 11:55:00,-1
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,2008-07-19 12:32:00,-1
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,2008-07-19 13:17:00,1


### 라벨 분포 및 결측치 개수 확인

In [6]:
# 라벨 분포 확인
print("🔎 Label distribution:")
print(eda_df['label'].value_counts())

# 결측치 개수 확인
nan_counts = eda_df.isna().sum()
nan_ratio = eda_df.isna().mean()

print("\n🔎 NaN overview:")
print(f"총 결측치 개수: {int(nan_counts.sum())}")
print(f"센서별 평균 결측 비율: {nan_ratio.mean():.4f}")

# 센서별 결측치 상위 10개
print("\n🔎 NaN ratio (Top 10 sensors):")
print(nan_ratio.sort_values(ascending=False).head(10))

🔎 Label distribution:
label
-1    1463
 1     104
Name: count, dtype: int64

🔎 NaN overview:
총 결측치 개수: 41951
센서별 평균 결측 비율: 0.0452

🔎 NaN ratio (Top 10 sensors):
sensor_158    0.911934
sensor_293    0.911934
sensor_294    0.911934
sensor_159    0.911934
sensor_493    0.855775
sensor_359    0.855775
sensor_086    0.855775
sensor_221    0.855775
sensor_247    0.649649
sensor_110    0.649649
dtype: float64
